### solve the local sequence alignment problem using smith-waterman algorithm

In [ ]:
import numpy as np
equal_score = 1
unequal_score = -1
space_score = -2
# smith-waterman 算法不可能出现负分的情况

In [54]:
def createScoreMatrix(list1, list2, debug=False):
    lenList1, lenList2 = len(list1), len(list2)
    #initialize matrix
    scoreMatrix = np.zeros((lenList1+1, lenList2+1), dtype=int)
    #populate the matrix
    for i, x in enumerate(list1):
        for j, y in enumerate(list2):
            if x == y:
                scoreMatrix[i+1][j+1] = scoreMatrix[i][j]+equal_score
            else:
                scoreMatrix[i+1][j+1] = max(scoreMatrix[i][j+1]+space_score, scoreMatrix[i+1][j]+space_score, scoreMatrix[i][j]+unequal_score, 0)
    if debug:
        print("score Matrix:")
        print(scoreMatrix)
    return scoreMatrix

list1=[1, 2, 4, 6,7,8,0]
list2=[4,5,7,1,2,0]
print(createScoreMatrix(list1, list2))

[[0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 2 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]]


In [77]:
def getMaxScorePosition(scoreMatrix):
    maxValue = np.amax(scoreMatrix)
    if maxValue == 0:
        return None
    else:
        return np.argwhere(scoreMatrix == maxValue)
    
list1=list("GGTTGACTA")
list2=list("TGTTACGG")
getMaxScorePosition(createScoreMatrix(list1, list2))

array([[4, 4],
       [7, 6]], dtype=int64)

In [91]:
def traceBack(list1, list2, scoreMatrix, startPosition):
    '''
    Return:
         alignedList1, alignedList2, commonSub
    '''
    commonSub=[]
    alignedList1 = []
    alignedList2 = []
    i, j = startPosition[0], startPosition[1]
    if i == 0 or j == 0:
        return list1, list2, commonSub
    else:
        #TODO:填充从结尾到最大值位置的元素或空格
        while i != 0 and j != 0 and scoreMatrix[i][j] > 0:  #顺序是左上，上，左
            if scoreMatrix[i][j] == scoreMatrix[i-1][j-1] + equal_score:
                commonSub.append(list1[i-1])
                alignedList1.append(list1[i-1])
                alignedList2.append(list2[j-1])
                i -= 1
                j -= 1
            elif scoreMatrix[i][j] == scoreMatrix[i-1][j-1] + unequal_score:
                alignedList1.append(list1[i-1])
                alignedList2.append(list2[j-1])
                i -= 1
                j -= 1
            elif scoreMatrix[i][j] == scoreMatrix[i-1][j] + space_score:
                alignedList1.append(list1[i-1])
                alignedList2.append('_')
                i -= 1
            else:#scoreMatrix[i][j] == scoreMatrix[i][j-1] + space_score:
                alignedList1.append('_')
                alignedList2.append(list2[j-1])
                j -= 1
        #TODO:填充从开始到回溯结束位置的元素或空格。
    alignedList1.reverse()
    alignedList2.reverse()
    commonSub.reverse()
    
    return alignedList1, alignedList2, commonSub

In [97]:
def smith_waterman(list1, list2, debug=False):
    scoreMatrix = createScoreMatrix(list1, list2, debug)
    startPositions = getMaxScorePosition(scoreMatrix)
    alignedList1s = []
    alignedList2s = []
    commonSubs = []
    if startPositions is None:
        return alignedList1s, alignedList2s, commonSubs
    for i in range(startPositions.shape[0]):
        alignedList1, alignedList2, commonSub = traceBack(list1, list2, scoreMatrix, startPositions[i])
        alignedList1s.append(alignedList1)
        alignedList2s.append(alignedList2)
        commonSubs.append(commonSub)
    return alignedList1s, alignedList2s, commonSubs

In [109]:
list1=list("GGTTGACTA")
list2=list("TGTTACGG")
alignedList1s,alignedList2s,commonSubs= smith_waterman(list1, list2, True)
print("There are %s commonSubs found.\n"% (len(alignedList1s)))
for i in range(len(alignedList1s)):
    print(alignedList1s[i])
    print(alignedList2s[i])
    print(commonSubs[i])

score Matrix:
[[0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 0 0 1 2]
 [0 1 0 2 1 0 0 0 0]
 [0 1 0 1 3 1 0 0 0]
 [0 0 2 0 1 2 0 1 1]
 [0 0 0 1 0 2 1 0 0]
 [0 0 0 0 0 0 3 1 0]
 [0 1 0 1 1 0 1 2 0]
 [0 0 0 0 0 2 0 0 1]]
There are 2 commonSubs found.

['G', 'T', 'T']
['G', 'T', 'T']
['G', 'T', 'T']
['G', 'T', 'T', 'G', 'A', 'C']
['G', 'T', 'T', '_', 'A', 'C']
['G', 'T', 'T', 'A', 'C']


In [108]:
list1=[1, 2, 4, 6,7,8,0]
list2=[4,5,7,1,2,0]
alignedList1s,alignedList2s,commonSubs= smith_waterman(list1, list2, True)
print("There are %s commonSubs found.\n"% (len(alignedList1s)))
for i in range(len(alignedList1s)):
    print(alignedList1s[i])
    print(alignedList2s[i])
    print(commonSubs[i])

score Matrix:
[[0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 2 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]]
There are 1 commonSubs found.

[1, 2]
[1, 2]
[1, 2]


In [107]:
list1=list("GCCCTAGCG")
list2=list("GCGCAATG")
alignedList1s,alignedList2s,commonSubs= smith_waterman(list1, list2, True)
print("There are %s commonSubs found.\n"% (len(alignedList1s)))
for i in range(len(alignedList1s)):
    print(alignedList1s[i])
    print(alignedList2s[i])
    print(commonSubs[i])

score Matrix:
[[0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 1]
 [0 0 2 0 2 0 0 0 0]
 [0 0 1 1 1 1 0 0 0]
 [0 0 1 0 2 0 0 0 0]
 [0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 1 2 0 0]
 [0 1 0 1 0 0 0 1 1]
 [0 0 2 0 2 0 0 0 0]
 [0 1 0 3 1 1 0 0 1]]
There are 1 commonSubs found.

['G', 'C', 'G']
['G', 'C', 'G']
['G', 'C', 'G']


In [106]:
list1=list("awefasdfef")
list2=list("23474565")
alignedList1s,alignedList2s,commonSubs= smith_waterman(list1, list2, True)
print("There are %s commonSubs found.\n"% (len(alignedList1s)))
for i in range(len(alignedList1s)):
    print(alignedList1s[i])
    print(alignedList2s[i])
    print(commonSubs[i])


score Matrix:
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
There are 0 commonSubs found.

